In [1]:
from clase_recomendacion import recomendacion
import pandas as pd
from google.cloud import bigquery
import spacy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import math
import numpy as np

In [14]:
def load_model(model_name): 
        # Intenta cargar el modelo
        try:
            nlp = spacy.load(model_name)
            return nlp
        except OSError:
            print(f"El modelo {model_name} no está instalado. Descargando e instalando...")
            spacy.cli.download(model_name)
            nlp = spacy.load(model_name)
            return nlp

In [15]:
nlp_en =  load_model("en_core_web_md")

***

In [2]:
recom = recomendacion()

In [4]:
recom.main('i like sushi and beer',(44.3069541, -89.8457834))


,name,address,distance
0,Forbidden Planet,111 S Dubuque St,324.79
1,Sushi House,17265 Chesterfield Airport Rd,630.03
2,Firehouse Pizza & Ribs,8611 Ridge Rd,738.38
3,Beer 4 U Sports Bar,2177 Crain Hwy,1247.51
4,Bluefin Express,1449 NW 51st St,2174.71


***

In [19]:
df = pd.read_pickle('../Datasets_ML/Rest_completo_ML.pickle')
df.head()

,business_id,name,address,city,coord,keywords,embedding
0,0x872d8f6b3060731f:0xbab7d25cfd777e60,Paleo Brio Healthy Kitchen-Flagstaff,"CWWX+P2 Parks, AZ, USA",Parks,"[35.4468219, -112.0524234]","[american restaurant, takeout, delivery, good ...","[[-0.99531996, -1.1494, -2.63405, -0.592645, 4..."
1,0x872a4991146626e7:0x7d5c5ed7e1a58fa3,Santan Barbecue & Grille,"X87J+3X Casa Grande, AZ, USA",Casa Grande,"[32.9626935, -111.6675081]","[barbecue restaurant, delivery, good for kids,...","[[0.21536501, -0.36992, -1.88865, 0.36057, 2.2..."
2,0x89e36d1b29148441:0xf577f42878da4838,El Pulgarcito,"F2HM+5G Lynn, MA, USA",Lynn,"[42.4779416, -70.966168]",[latin american restaurant],"[[-0.69983, -0.58922666, 0.42566672, -0.123566..."
3,0x89a9f643802fd7bf:0xd4948847f210b7cf,Subway,"44W3+J3 Silver Lake, NC, USA",Silver Lake,"[34.1465696, -77.8973559]","[fast food restaurant, delivery, takeout, dine...","[[2.6815202, -1.0777999, -5.9206333, 0.7224250..."
4,0x80c8dcfe21b0ac85:0x1f1ba837d0fb5612,Champion's Grille at Sunrise Vista,"6W6X+FJ Sunrise Manor, NV, USA",Sunrise Manor,"[36.2111922, -115.050931]","[hamburger restaurant, outdoor seating, takeou...","[[0.94348, -0.88408, -2.66125, 0.18630001, 2.4..."


In [23]:
keywords = recom.keywords('I like sushi and beer')
indices = recom.search(keywords)
df_recom = recom.df.iloc[indices].reset_index()
df_recom['distance'] = df_recom.apply(lambda row: recom.distance([44.3069541, -89.8457834], row['coord']),axis=1)
df_recom['distance'] = df_recom['distance'].round(2)
df_recom = df_recom.sort_values(by='distance', ascending=True).reset_index().head(5)
df_recom[['name', 'address', 'distance']]

,name,address,distance
0,Subway,9630 State Hwy 13 South,2.81
1,Subway,106 1st St,3.62
2,Jabberjawz Bar And Dining,3704 WI-54,6.40
3,Domino's Pizza,2880 8th St S,7.21
4,Tom's on Grand,410 E Grand Ave,9.35


In [16]:
keywords = recom.keywords('I like sushi and beer')
keywords

['sushi', 'beer']

In [17]:
embeddings = np.array([nlp_en(palabra).vector for palabra in keywords])
embeddings

array([[-4.0768e-01, -2.5913e+00, -2.6877e+00,  1.2205e+00,  8.8824e-01,
        -9.1448e-01, -2.0154e+00, -1.6040e+00, -3.7208e+00,  1.0235e+00,
         4.1116e+00,  1.3711e+00,  5.2261e+00,  3.8185e-01,  1.7290e+00,
        -2.5817e+00,  1.3448e+00, -9.9214e-01,  1.2694e-01, -1.9569e+00,
         1.3648e+00,  1.8844e+00, -8.0763e-01,  7.8780e-01,  1.3465e+00,
         9.9532e-01, -3.5990e+00, -1.2769e+00, -3.8955e+00, -3.6188e+00,
         1.2644e+00, -1.8774e+00,  6.3247e-01, -1.4067e-01,  1.7372e+00,
         4.4825e-01, -4.2351e-01,  7.7238e-02, -1.3619e+00,  7.9554e-01,
        -9.6348e-01, -1.0216e+00,  1.3510e+00, -2.5703e-02,  4.6945e+00,
        -6.3223e-01, -1.0991e+00, -2.3394e+00,  8.3488e-01,  3.4334e-01,
        -3.9493e+00, -1.3800e+00,  4.5980e+00, -3.0030e+00,  1.5122e+00,
        -2.2479e+00, -2.2306e-01, -2.2357e-01,  9.9911e-01,  9.1581e-01,
         2.5501e+00, -8.9346e-01,  4.9703e-01,  7.7197e-01, -1.5350e+00,
         1.8196e+00, -2.7875e+00,  3.2578e-01,  6.1

In [7]:
df_recom['keywords'][0]

['barbecue restaurant',
 'delivery',
 'good for kids',
 'wheelchair accessible entrance',
 'lunch',
 'dinner',
 'solo dining',
 'alcohol',
 'beer',
 'casual',
 'debit cards']

In [7]:
df_recom

,index,business_id,name,address,city,coord,keywords,embedding
0,3,0x89a9f643802fd7bf:0xd4948847f210b7cf,Subway,"44W3+J3 Silver Lake, NC, USA",Silver Lake,"[34.1465696, -77.8973559]","[fast food restaurant, delivery, takeout, dine...","[[2.6815202, -1.0777999, -5.9206333, 0.7224250..."
1,4,0x80c8dcfe21b0ac85:0x1f1ba837d0fb5612,Champion's Grille at Sunrise Vista,"6W6X+FJ Sunrise Manor, NV, USA",Sunrise Manor,"[36.2111922, -115.050931]","[hamburger restaurant, outdoor seating, takeou...","[[0.94348, -0.88408, -2.66125, 0.18630001, 2.4..."
2,5,0x89de0a23534d2cc9:0xd2f2aaf44db2dc65,Pho Yum at Empire State Plaza,"M62R+35 Albany, NY, USA",Albany,"[42.6501296, -73.75955309999999]","[vietnamese restaurant, takeout, dine-in, deli...","[[-1.13372, -1.3051, -2.5974002, 0.791155, 4.7..."
3,6,0x874d9a9cf977b1cf:0x3e52739289478daa,Bandera Brisket,"5FG7+RM Springville, UT, USA",Springville,"[40.1770645, -111.5357944]","[barbecue restaurant, takeout, delivery, good ...","[[0.21536501, -0.36992, -1.88865, 0.36057, 2.2..."
4,7,0x89b6560c4d0377d1:0xf7580b9be88529b,Burger King,"JPV4+92 Dale City, VA, USA",Dale City,"[38.643406, -77.29496]","[fast food restaurant, no-contact delivery, de...","[[2.6815202, -1.0777999, -5.9206333, 0.7224250..."
...,...,...,...,...,...,...,...,...
32518,53485,0x8883c1a8c3e5a875:0x5a5d602f4bd9f687,Subway,1002 W Beacon St,Philadelphia,"[32.7674868, -89.1276494]","[fast food restaurant, curbside pickup, delive...","[[2.6815202, -1.0777999, -5.9206333, 0.7224250..."
32519,53486,0x8883c1b9e197cab3:0x3c46d7d4cfb5cb7f,Penn's Fish House LLC,257 W Beacon St #3154,Philadelphia,"[32.7728103, -89.1170877]","[seafood restaurant, takeout, delivery, good f...","[[0.93588006, -2.87195, -2.29095, 0.764905, 2...."
32520,53487,0x8883c1b0a4a5c08d:0x3cc1dc6b505d482,Loaves & Fishes,344 W Main St,Philadelphia,"[32.770609799999995, -89.11920649999999]","[seafood restaurant, takeout, delivery, good f...","[[0.93588006, -2.87195, -2.29095, 0.764905, 2...."
32521,53488,0x8883c1ba01ade5e9:0x7e6ae697de13b254,Laredo Grill,230 Canal Pl,Philadelphia,"[32.7636062, -89.1398403]","[mexican restaurant, takeaway, dine-in, delive...","[[-1.15987, -0.23539498, -2.703, -0.27396, 4.7..."


In [21]:
df

,business_id,name,address,city,coord,keywords,embedding
0,0x872d8f6b3060731f:0xbab7d25cfd777e60,Paleo Brio Healthy Kitchen-Flagstaff,"CWWX+P2 Parks, AZ, USA",Parks,"[35.4468219, -112.0524234]","[american restaurant, takeout, delivery, good ...","[[-0.99531996, -1.1494, -2.63405, -0.592645, 4..."
1,0x872a4991146626e7:0x7d5c5ed7e1a58fa3,Santan Barbecue & Grille,"X87J+3X Casa Grande, AZ, USA",Casa Grande,"[32.9626935, -111.6675081]","[barbecue restaurant, delivery, good for kids,...","[[0.21536501, -0.36992, -1.88865, 0.36057, 2.2..."
2,0x89e36d1b29148441:0xf577f42878da4838,El Pulgarcito,"F2HM+5G Lynn, MA, USA",Lynn,"[42.4779416, -70.966168]",[latin american restaurant],"[[-0.69983, -0.58922666, 0.42566672, -0.123566..."
3,0x89a9f643802fd7bf:0xd4948847f210b7cf,Subway,"44W3+J3 Silver Lake, NC, USA",Silver Lake,"[34.1465696, -77.8973559]","[fast food restaurant, delivery, takeout, dine...","[[2.6815202, -1.0777999, -5.9206333, 0.7224250..."
4,0x80c8dcfe21b0ac85:0x1f1ba837d0fb5612,Champion's Grille at Sunrise Vista,"6W6X+FJ Sunrise Manor, NV, USA",Sunrise Manor,"[36.2111922, -115.050931]","[hamburger restaurant, outdoor seating, takeou...","[[0.94348, -0.88408, -2.66125, 0.18630001, 2.4..."
...,...,...,...,...,...,...,...
53485,0x8883c1a8c3e5a875:0x5a5d602f4bd9f687,Subway,1002 W Beacon St,Philadelphia,"[32.7674868, -89.1276494]","[fast food restaurant, curbside pickup, delive...","[[2.6815202, -1.0777999, -5.9206333, 0.7224250..."
53486,0x8883c1b9e197cab3:0x3c46d7d4cfb5cb7f,Penn's Fish House LLC,257 W Beacon St #3154,Philadelphia,"[32.7728103, -89.1170877]","[seafood restaurant, takeout, delivery, good f...","[[0.93588006, -2.87195, -2.29095, 0.764905, 2...."
53487,0x8883c1b0a4a5c08d:0x3cc1dc6b505d482,Loaves & Fishes,344 W Main St,Philadelphia,"[32.770609799999995, -89.11920649999999]","[seafood restaurant, takeout, delivery, good f...","[[0.93588006, -2.87195, -2.29095, 0.764905, 2...."
53488,0x8883c1ba01ade5e9:0x7e6ae697de13b254,Laredo Grill,230 Canal Pl,Philadelphia,"[32.7636062, -89.1398403]","[mexican restaurant, takeaway, dine-in, delive...","[[-1.15987, -0.23539498, -2.703, -0.27396, 4.7..."
